In [422]:
from reader import ebook
from main import read_list,read_json
from typing import Optional
import requests
from dotenv import load_dotenv
import json

In [136]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"


In [486]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and easy to parse.)
Input:  
1.Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
3.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize the chapter, integrating context and key developments from previous chapters and create a self containing summary. end with to be continued.
2.Character List: Update existing character descriptions or add new characters based on this chapter. If no characters are mentioned, return the same as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Summary of the current chapter in context of previous chapters.",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}

example:
{
  "summary": "In this chapter, Mira embarks on a courageous journey to seek the sun goddess Solara, facing numerous challenges along the way. As she navigates through the enchanted forest, her determination shines through, reflecting the hope of her village. The chapter highlights her growth and resilience against the encroaching darkness.",
  "characters": {
    "Mira": "A 16-year-old girl with long, wavy black hair and bright green eyes. She wears a simple tunic made of earth-toned fabric, suitable for travel, and has a determined expression. Mira carries a small satchel with essential supplies and exhibits adventurous body language, often standing tall with her shoulders back."
  },
  "places": {
    "Enchanted Forest": "A mystical area filled with vibrant flora and magical creatures, bathed in dappled sunlight. The air is rich with the sounds of nature, creating an atmosphere of wonder and adventure."
  }
}

'''


In [423]:
def msg(text: str, context: str = "Its first chapter", characters: dict = {}, places: dict = {}) -> list:
    js=json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places})
    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

In [424]:
def get_scene(messages) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.5,
        "stream": False,
        "max_tokens":10000
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [487]:
book=ebook("./books/LP.epub")
chapters = book.get_chapters()
_,text=zip(*chapters[:5]) 



In [488]:

output_dict={0:{"summary":"This is the first chapter","characters":{},"places": {}}
}

In [489]:
for idx,i in enumerate(text):
    mes=msg(text,output_dict[idx]["summary"])
    summary_characters=get_scene(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    

chapter: 0 done
chapter: 1 done
chapter: 2 done
chapter: 3 done
chapter: 4 done


In [ ]:
output_dict

In [490]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:

Places:
Sahara Desert: A vast, isolated desert region where the chapter's key events take place.

Summary:
The chapter begins with the narrator recalling a childhood memory of a drawing he made of a boa constrictor digesting an elephant, which was met with confusion by adults. He then recounts a later encounter in the Sahara desert with a small, enigmatic figure who requests the narrator to draw him a sheep. Despite the narrator's initial struggles, he eventually draws a sheep to the little figure's satisfaction. The chapter ends without revealing the true nature of the little figure.

-------------------------------------------------------------------------------------------------------------------
CHAPTER 2 :

Characters:

Places:
Sahara Desert: A vast, isolated desert area in North Africa. The setting is at night, with the narrator stranded due to a plane breakdown.

Summary:
In this chapter, the narrator recounts a childhood memory of a drawing he made of 

In [406]:
b=read_json(a)

In [407]:
b

{'narrative_summary': 'The narrator recounts his past experiences, including the loss of his artistic dreams due to the lack of understanding from adults, and his encounter with a mysterious voice in the Sahara Desert that asked him to draw a sheep. This encounter rekindled his connection to childhood innocence.',
 'characters': {'The Narrator': 'An unnamed individual who is a seasoned traveler and pilot. He has a cynical view of adults and their lack of imagination. He carries a drawing of a mysterious little man he encountered in the Sahara Desert.'}}

In [402]:
char=b['characters']
summary=b['narrative_summary']

In [404]:
print("characters:")
for name,disc in char.items():
    print(f"{name}:\n{disc}")
print(f"\nSummary:\n{summary}")


characters:
The Narrator:
An unnamed character who is a grown-up, a pilot, and an artist at heart. He has a pessimistic view of adults and their lack of imagination. He wears a pilot's uniform and has a serious expression.

Summary:
The narrator reflects on his past, his lost artistic dreams, and his encounter with a mysterious voice in the Sahara Desert. This voice, later identified as the little prince, asks him to draw a sheep, reigniting his connection to childhood innocence.
